##### 版權所有 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemma - RAG with ChromaDB

這本 Cookbook 展示了如何在不使用任何協作工具（如 LangChain 或 LlamaIndex）的情況下建構一個簡約的檢索增強生成（RAG）系統。它僅使用 [ChromaDB](https://www.trychroma.com/) 作為儲存和查詢嵌入的向量資料庫。

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/gemma-cookbook/blob/zh-tw-240628/Gemma/RAG_with_ChromaDB.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## 設定

### 選擇 Colab 執行環境
要完成此指南，你需要有一個具有足夠資源來執行 Gemma 模型的 Colab 執行環境。在這種情況下，你可以使用 T4 GPU：

1. 在 Colab 視窗的右上角，選擇 **▾ (其他連接選項)** 。
2. 選擇 **變更執行環境類型** 。
3. 在 **硬體加速器** 下，選擇 **T4 GPU** 。

### 在 Hugging Face 上設定 Gemma
這本 Cookbook 使用 Hugging Face 上經過指令調整的 Gemma 7B 模型。因此你需要：

* 通過接受特定模型的 Hugging Face 頁面上的 Gemma 授權來獲取 [huggingface.co](huggingface.co) 上的 Gemma 訪問權限，即 [Gemma 7B IT](https://huggingface.co/google/gemma-7b-it)。
* 生成一個 [Hugging Face 訪問令牌](https://huggingface.co/docs/hub/en/security-tokens) 並將其配置為 Colab 的秘密 'HF_TOKEN'。


## 檢索增強生成 (RAG)

大型語言模型 (LLM) 可以在未經直接訓練的情況下學習新能力。然而，LLM 在被要求提供其未經訓練的問題的回應時，已知會出現“幻覺”。這部分是因為 LLM 在訓練後對事件一無所知。也很難追溯 LLM 從中提取回應的來源。對於可靠、可延展的應用程式，重要的是 LLM 提供基於事實的回應並能夠引用其資訊來源。

克服這些限制的一種常見方法稱為檢索增強生成 (RAG)，它通過資訊檢索 (IR) 機制從外部知識庫檢索相關數據來增強發送給 LLM 的提示。知識庫可以是你自己的文件、資料庫或 API。

### 將數據分塊

為了提高在檢索期間向向量資料庫返回的內容的相關性，在攝取文件時將大型文件分解為較小的部分或塊。

在這本 Cookbook 中，你將使用 [Google I/O 2024 Gemma 家族擴展發佈博客](https://developers.googleblog.com/en/gemma-family-and-toolkit-expansion-io-2024/) 作為範例文件，並使用 Google 的 [Open Source HtmlChunker](https://github.com/google/labs-prototypes/tree/main/seeds/chunker-python) 將其分塊為段落。


In [ ]:
!pip install google-labs-html-chunker

from google_labs_html_chunker.html_chunker import HtmlChunker

from urllib.request import urlopen

with urlopen(
    "https://developers.googleblog.com/en/gemma-family-and-toolkit-expansion-io-2024/"
) as f:
    html = f.read().decode("utf-8")

# Chunk the file using HtmlChunker
chunker = HtmlChunker(
    max_words_per_aggregate_passage=200,
    greedily_aggregate_sibling_nodes=True,
    html_tags_to_exclude={"noscript", "script", "style"},
)
passages = chunker.chunk(html)

看看分塊文字的樣子。


In [ ]:
for passage in passages:
    print(passage)

Introducing PaliGemma, Gemma 2, and an Upgraded Responsible AI Toolkit
            
            
            
            - Google Developers Blog
Products Develop Android Chrome ChromeOS Cloud Firebase Flutter Google Assistant Google Maps Platform Google Workspace TensorFlow YouTube Grow Firebase Google Ads Google Analytics Google Play Search Web Push and Notification APIs Earn AdMob Google Ads API Google Pay Google Play Billing Interactive Media Ads Solutions Events Learn Community Groups Google Developer Groups Google Developer Student Clubs Woman Techmakers Google Developer Experts Tech Equity Collective Programs Accelerator Solution Challenge DevFest Stories All Stories Developer Profile Blog Search English English Español (Latam) Bahasa Indonesia 日本語 한국어 Português (Brasil) 简体中文
Products More Solutions Events Learn Community More Developer Profile Blog Develop Android Chrome ChromeOS Cloud Firebase Flutter Google Assistant Google Maps Platform Google Workspace TensorFlow YouTube G

## 使用向量資料庫索引這些塊


你現在將使用 ChromaDB，一個開放原始碼嵌入式資料庫，來索引這些段落。


In [ ]:
!pip install chromadb
import chromadb

chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="cookbook_collection")
collection.add(documents=passages, ids=[str(i) for i in range(len(passages))])

接下來，你會根據使用者問題從向量資料庫中檢索相關段落作為上下文，並使用使用者問題和檢索到的上下文組裝一個提示。


In [ ]:
prompt_template = """You are an expert in answering user questions. You always understand user questions well, and then provide high-quality answers based on the information provided in the context.

If the provided context does not contain relevent information, just respond "I could not find the answer based on the context you provided."

User question: {}

Context:
{}
"""

user_question = "how many parameters does Gemma 2 have?"

results = collection.query(query_texts=user_question, n_results=3)

context = "\n".join(
    [f"{i+1}. {passage}" for i, passage in enumerate(results["documents"][0])]
)
prompt = f"{prompt_template.format(user_question, context)}"

這是將發送給 Gemma 的最終提示。


In [ ]:
print(prompt)

You are an expert in answering user questions. You always understand user questions well, and then provide high-quality answers based on the information provided in the context.

If the provided context does not contain relevent information, just respond "I could not find the answer based on the context you provided."

User question: how many parameters does Gemma 2 have?

Context:
1. Gemma 2 is still pretraining. This chart shows performance from the latest Gemma 2 checkpoint along with benchmark pretraining metrics. Source: Hugging Face Open LLM Leaderboard (April 22, 2024) and Grok announcement blog
2. Stay tuned for the official launch of Gemma 2 in the coming weeks! Expanding the Responsible Generative AI Toolkit For this reason we're expanding our Responsible Generative AI Toolkit to help developers conduct more robust model evaluations by releasing the LLM Comparator in open source. The LLM Comparator is a new interactive and visual tool to perform effective side-by-side evaluat

### 生成答案

現在使用 Hugging Face 以量化 4-bit 模式載入 Gemma 模型。


In [ ]:
!pip install bitsandbytes accelerate
from transformers import AutoTokenizer
import transformers
import torch
import bitsandbytes, accelerate

model = "google/gemma-1.1-7b-it"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
    },
)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

最終，生成答案。


In [ ]:
messages = [
    {"role": "user", "content": prompt},
]
prompt = pipeline.tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.1)
print(outputs[0]["generated_text"][len(prompt) :])

Gemma 2 has **27 billion parameters**.

The context explicitly states that Gemma 2 has 27 billion parameters.


Gemma 能夠根據上下文提供正確的答案。
